In [14]:
import numpy as np

# Fonctions d'activation
def sigmoid(x):
    result = 1 / (1 + np.exp(-np.clip(x, -500, 500))) # Clipping pour stabilité
    assert np.all((result >= 0) & (result <= 1)), "Sigmoïde doit être entre 0 et 1"
    return result

def sigmoid_derivative(x):
    s = sigmoid(x)
    result = s * (1 - s)
    assert np.all(result >= 0), "Dérivée de la sigmoïde doit être >= 0"
    if np.isscalar(x) and x == 0:
        assert np.isclose(result, 0.25), "Dérivée de la sigmoïde à 0 doit être 0.25"
    return result

def relu(x):
    result = np.maximum(0, x)
    assert np.all(result >= 0), "ReLU doit être >= 0"
    return result

def relu_derivative(x):
    result = np.where(x > 0, 1, 0)
    assert np.all((result == 0) | (result == 1)), "Dérivée de ReLU doit être 0 ou 1"
    return result

# Classe Layer
class Layer:
    def __init__(self, input_size, output_size, activation, seed=None):
        if seed is not None:
            np.random.seed(seed)
        # Initialisation adaptée à l'activation
        if activation == 'sigmoid':
            # Initialisation Xavier
            self.W = np.random.randn(input_size, output_size) * np.sqrt(1. / input_size)
        elif activation == 'relu':
            # Initialisation He
            self.W = np.random.randn(input_size, output_size) * np.sqrt(2. / input_size)
        else:
            raise ValueError("Activation non supportée")
        self.b = np.zeros((1, output_size))
        self.activation = activation
        # Assertions pour vérifier les dimensions
        assert self.W.shape == (input_size, output_size), "Mauvaise dimension pour W"
        assert self.b.shape == (1, output_size), "Mauvaise dimension pour b"

    def forward(self, X):
        self.X = X
        self.Z = np.dot(X, self.W) + self.b
        if self.activation == 'sigmoid':
            self.A = sigmoid(self.Z)
        elif self.activation == 'relu':
            self.A = relu(self.Z)
        else:
            raise ValueError("Activation non supportée")
        assert self.A.shape == (X.shape[0], self.W.shape[1]), "Mauvaise dimension pour A"
        return self.A

    def backward(self, dA, learning_rate):
        if self.activation == 'sigmoid':
            dZ = dA * sigmoid_derivative(self.Z)
        elif self.activation == 'relu':
            dZ = dA * relu_derivative(self.Z)
        else:
            raise ValueError("Activation non supportée")
        
        # Clipping des gradients pour stabilité
        dZ = np.clip(dZ, -100, 100)
        
        m = self.X.shape[0]
        dW = (1 / m) * np.dot(self.X.T, dZ)
        db = (1 / m) * np.sum(dZ, axis=0, keepdims=True)
        
        # Assertions pour vérifier les dimensions
        assert dW.shape == self.W.shape, "Mauvaise dimension pour dW"
        assert db.shape == self.b.shape, "Mauvaise dimension pour db"
        
        # Mise à jour des paramètres
        self.W -= learning_rate * dW
        self.b -= learning_rate * db
        
        # Gradient pour la couche précédente
        dA_prev = np.dot(dZ, self.W.T)
        return dA_prev

# Classe Model
class Model:
    def __init__(self, layers, learning_rate=0.1):
        self.layers = layers
        self.lr = learning_rate

    def forward(self, X):
        A = X
        for layer in self.layers:
            A = layer.forward(A)
        return A

    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        loss = np.mean((y_true - y_pred) ** 2)
        assert loss >= 0, "La perte doit être positive ou nulle"
        if np.array_equal(y_true, y_pred):
            assert np.isclose(loss, 0), "La perte doit être 0 si y_true = y_pred"
        return loss

    def backward(self, X, y, y_pred):
        m = X.shape[0]
        dA = (y_pred - y) # Gradient initial pour la couche de sortie
        for layer in reversed(self.layers):
            dA = layer.backward(dA, self.lr)

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.compute_loss(y, y_pred)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")
            self.backward(X, y, y_pred)

    def test(self, X, y):
        predictions = self.forward(X)
        print("\nPrédictions après entraînement :")
        for i in range(len(X)):
            print(f"Entrée: {X[i]}, Prédiction: {predictions[i][0]:.4f}, Attendu: {y[i][0]}")
            if y[i][0] == 0:
                assert predictions[i][0] < 0.5, f"Prédiction pour {X[i]} devrait être < 0.5"
            else:
                assert predictions[i][0] > 0.5, f"Prédiction pour {X[i]} devrait être > 0.5"

# Test du modèle
if __name__ == "__main__":
    # Données XOR
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y = np.array([[0], [1], [1], [0]])

    # Création des couches
    layers = [
        Layer(input_size=2, output_size=4, activation='sigmoid', seed=42),
        Layer(input_size=4, output_size=3, activation='relu', seed=42),
        Layer(input_size=3, output_size=1, activation='sigmoid', seed=42)
    ]

    # Création et entraînement du modèle
    model = Model(layers, learning_rate=0.5)
    model.train(X, y, epochs=10000)
    model.test(X, y)

Epoch 0, Loss: 0.263327211206297
Epoch 100, Loss: 0.2496567741445596
Epoch 200, Loss: 0.24955228505461724
Epoch 300, Loss: 0.24943098085528878
Epoch 400, Loss: 0.2492851914829899
Epoch 500, Loss: 0.2491046665168712
Epoch 600, Loss: 0.2488751959696997
Epoch 700, Loss: 0.24857656177916976
Epoch 800, Loss: 0.24817943349167876
Epoch 900, Loss: 0.24764058730481286
Epoch 1000, Loss: 0.246895389052751
Epoch 1100, Loss: 0.24584558225684133
Epoch 1200, Loss: 0.2443385110262897
Epoch 1300, Loss: 0.24214431614475068
Epoch 1400, Loss: 0.2389014779629115
Epoch 1500, Loss: 0.23402975370357335
Epoch 1600, Loss: 0.22666371088333848
Epoch 1700, Loss: 0.21574924200495282
Epoch 1800, Loss: 0.199947589419715
Epoch 1900, Loss: 0.1758876317654965
Epoch 2000, Loss: 0.13355890592074504
Epoch 2100, Loss: 0.07258987450282711
Epoch 2200, Loss: 0.03222877167510081
Epoch 2300, Loss: 0.017241782254947203
Epoch 2400, Loss: 0.011494864682969515
Epoch 2500, Loss: 0.008171515142374278
Epoch 2600, Loss: 0.00627270493240